
# Indexing Techniques and Data Structures.

After learning about hashing functions and techniques, we will be exploring an implementation of such techniques called bloom filters. Firstly we will briefly discuss the [standard bloom filter](https://en.wikipedia.org/wiki/Bloom_filter), then improve upon it using [counting bloom filters](https://en.wikipedia.org/wiki/Counting_Bloom_filter).

## Standard Bloom filters

According to [Wikipedia](https://www.wikipedia.org) a [bloom filter](https://en.wikipedia.org/wiki/Bloom_filter) is "a space-efficient probabilistic data structure" conceived by [Burton Howard Bloom](https://en.wikipedia.org/wiki/Burton_Howard_Bloom) in 1970 that "tests whether an element is a member of a set". ([Bloom Filters - Geeks for Geeks][geeksforgeeks]

James Graham, an engineer at Medium, mentions that they are a quick way of "answering a YES/NO question". Although  in programming, we always have to work with _trade-offs_. They will "very rarely say YES when the answer is NO, but they will never say NO when the answer is actually YES".([What are Bloom filters?][medium]). Since they are probabilistic the more items we add into a bloom filter the higher the chance of getting a _false positive_.

Unlike a normal hash table it is an array of $m$ bits, initiated with all zeros, and requires $k$ hash functions to insert an element. The hash functions reveal the slots that are flipped from 0 to 1 during insertion of an element.

![Empty bloom filter](./images/empty_bloom.png "An empty bloom filter")

![Inserting elements into a bloom filter](./images/insert_bloom.png "Inserting elements into a bloom filter")

[medium]: https://blog.medium.com/what-are-bloom-filters-1ec2a50c68ff

[geeksforgeeks]: https://www.geeksforgeeks.org/bloom-filters-introduction-and-python-implementation/)


## Appendix

### Appendix 1 - Dashboard screenshot

![Dashboard Screenshot](./images/my_dashboard_feb24.png "Haitham Alhad's CS110 Dashboard screenshot taken on February 24th 2020")